In [118]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

In [119]:
url = 'https://crossfitjohnscreek.sites.zenplanner.com/calendar.cfm?DATE=2018-09-02&VIEW=list'

In [120]:
response = requests.get(url)

In [131]:
response.status_code

200

In [122]:
soup = BeautifulSoup(response.text, "html.parser")

In [137]:
week_sched = soup.findAll('table')[2]

In [149]:
week_sched.findAll('tr')

[<tr class="group">
 <td colspan="6">Sunday, September 2, 2018</td>
 </tr>,
 <tr class="item" onclick="checkLoggedId('enrollment.cfm?appointmentId=BD3848AD-1B6F-4E84-BA1D-8EED4CF379E0');">
 <td class="label"><div class="clickable hover-opacity-8 calendar-custom-color-d93836" style="padding:3px;">1:00 PM</div></td>
 <td class="items">CrossFit Open Gym - Unlimited members only (7/35)</td>
 <td class="items">CFJC Staff</td>
 <td class="items">Southern Strength and Conditioning</td>
 <td class="items">(28 spots left)</td>
 <td class="items"></td>
 </tr>,
 <tr class="group">
 <td colspan="6">Monday, September 3, 2018</td>
 </tr>,
 <tr class="item" onclick="checkLoggedId('enrollment.cfm?appointmentId=FE905FA5-6113-406E-A873-804F50E86492');">
 <td class="label"><div class="clickable hover-opacity-8 calendar-custom-color-17AD37" style="padding:3px;">9:30 AM</div></td>
 <td class="items">All Levels (18/70)</td>
 <td class="items">Dan Sturm</td>
 <td class="items">Southern Strength and Condition

In [205]:
time_format = '%A, %B %d, %Y, %I:%M %p'
weekly_attendance = []
for l in week_sched.find_all('tr'):
    if l.attrs['class'] == ['group']:
        date_str = l.find_next('td').text
    else:
        i = 0
        cell_data = {}
        for cell in l.find_all('td'):
            if cell.attrs['class'] == ["label"]:
                tod = cell.find_next('div').text
                date_str_full = date_str + ", " + tod
            else:
                if i == 1:
                    regex_match = re.match(r"(.*)\((\d+)", cell.text)
                    cell_data['session'] = regex_match.group(1)
                    cell_data['attendance'] = int(regex_match.group(2))
                elif i == 2:
                    cell_data['coach'] = cell.text
            i+=1
        ts = int(time.mktime(time.strptime(date_str_full, time_format)))
        cell_data['time_stamp'] = ts
        weekly_attendance.append(cell_data)
print(weekly_attendance)

[{'session': 'CrossFit Open Gym - Unlimited members only ', 'attendance': 7, 'coach': 'CFJC Staff', 'time_stamp': 1535907600}, {'session': 'All Levels ', 'attendance': 18, 'coach': 'Dan Sturm', 'time_stamp': 1535981400}, {'session': 'Mobility Monday Half Hour ', 'attendance': 0, 'coach': 'Ronda Smith', 'time_stamp': 1536006600}, {'session': 'Mobility Half Hour ', 'attendance': 0, 'coach': 'Ronda Smith', 'time_stamp': 1536013800}, {'session': 'CrossFIt - All Levels ', 'attendance': 8, 'coach': 'Ronda Smith', 'time_stamp': 1536051600}, {'session': 'CrossFIt - All Levels ', 'attendance': 1, 'coach': 'Ronda Smith', 'time_stamp': 1536055200}, {'session': 'Endurance WOD ', 'attendance': 2, 'coach': 'Lauren Cortjens', 'time_stamp': 1536055200}, {'session': 'CrossFIt - All Levels ', 'attendance': 7, 'coach': 'Ronda Smith', 'time_stamp': 1536060600}, {'session': 'CrossFIt - All Levels ', 'attendance': 3, 'coach': 'Ivan Terra', 'time_stamp': 1536064200}, {'session': 'CrossFIt - All Levels ', 'at

In [199]:
re.match(r"(.*)\((\d+)", "CrossFit Open Gym - Unlimited members only (7/35)").group(2)

'7'